In [1]:
# Calculate velocities at the sill
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os, sys
sys.path.append(os.path.abspath('./TEF'))
import TEF_Variables as tef

In [2]:
case_id = 5
file1 = xr.open_dataset('/work/oceans/wbao/MITgcm_results/iceplume/6_Tide_minhs/state_' + str(format(case_id,'03d')) + '.nc')
State = file1.isel(T=~file1.get_index("T").duplicated())
Grid = xr.open_dataset('/work/oceans/wbao/MITgcm_results/iceplume/6_Tide_minhs/grid_' + str(format(case_id,'03d')) + '.nc')



# Confine to the range of fjord
# state = State.isel(X=range(260), Xp1=range(261), Y=range(35,45), T=range(1,len(State.T)))
# grid = Grid.isel(X=range(260), Xp1=range(261), Y=range(35,45))

HFacC = Grid.HFacC
areaC = Grid.HFacC * Grid.drF * Grid.dyF # Z Y X
areaW = Grid.HFacW * Grid.drF * Grid.dyG # Z Y Xp1
sW = np.where(HFacC[-1,40,1:] == 0)[-1][0] # Sill west edge index
sE = np.where(HFacC[-1,40,1:] == 0)[-1][-1] # Sill east edge index
sC = np.ceil((sW+sE)/2).astype(int) # Sill crest index (178)
print(sC)

178


In [3]:
areaW.data[:,:,-2].sum(), areaC.data[:,:,-1].sum()

(21120000.0, 21120000.0)

In [4]:
# TEF calculation
state = State.isel(Y=range(35,45), T=range(1,len(State.T)))
grid = Grid.isel(Y=range(35,45))
s = state.S.data.mean(2) # Salinity in T,Z,X
u = (state.U.data[:,:,:,1:].mean(2) + state.U.data[:,:,:,:-1].mean(2)) / 2 # Along channel velocity
ot = state.T.data # Time in seconds


HFacC1 = grid.HFacC.data.mean(1)
dyF1 = grid.dyF.data.mean(0)
drF1 = grid.drF.data
gridA = np.broadcast_to(drF1[:, np.newaxis], HFacC1.shape) * np.broadcast_to(dyF1[np.newaxis, :], HFacC1.shape) * HFacC1 # Grid Area
da = np.broadcast_to(gridA[np.newaxis,:,:], u.shape)

S = state.S.data
U = (state.U.data[:,:,:,1:] + state.U.data[:,:,:,:-1]) / 2
drF = np.broadcast_to(grid.drF.data[np.newaxis, :, np.newaxis, np.newaxis], U.shape)
dyF = np.broadcast_to(grid.dyF.data[np.newaxis, np.newaxis, :, :], U.shape)
HFacC2 = np.broadcast_to(grid.HFacC.data[np.newaxis, :, :, :], U.shape)
DA = drF * dyF * HFacC2


tef_q1, tef_vel1, tef_da1, tef_qs1, tef_qs21, sbins1 = tef.process_section(U,S,DA,ot,sC,23,testing=False)
qin1, qout1, sin1, sout1 = tef.bulk_calc(tef_q1, tef_vel1, tef_da1, tef_qs1, tef_qs21, sbins1, ot)

  time 0 out of 120


/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:268: RuntimeWarning: invalid value encountered in true_divide
  Sp = QSp/Qp
/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:269: RuntimeWarning: invalid value encountered in true_divide
  Sm = QSm/Qm
/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:272: RuntimeWarning: invalid value encountered in true_divide
  S2p = QS2p/Qp
/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:273: RuntimeWarning: invalid value encountered in true_divide
  S2m = QS2m/Qm


In [5]:
# Velocities at the sill
U0 = 1e-3 # Tidal amplitude at OBC
Ut = U0 * areaC.data[:,:,-1].sum() / areaC.data[:,:,sC].sum() # Maximum tidal velocity at the sill

t0 = 89
Ue = (qin1[t0:].mean()-qout1[t0:].mean()) / (2*areaC.data[:,:,sC].sum()) # Exchange velocity at the sill

print(Ut,Ue)

0.66 0.07479216702738467


In [6]:
qin1[t0:].mean(), qout1[t0:].mean()

(2536.6221958531005, -2250.076493899518)

In [7]:
(qin1-qout1) / (2*areaC.data[:,:,sC].sum())

array([0.01319195, 0.01825821, 0.02557309, 0.03022576, 0.0347196 ,
       0.03780681, 0.03881877, 0.03882819, 0.03678939, 0.0336676 ,
       0.0288968 , 0.02665538, 0.02083433, 0.01788984, 0.0173277 ,
       0.00541886, 0.01544045, 0.01852391, 0.03453606, 0.03822816,
       0.0414521 , 0.04145663, 0.04215806, 0.04003236, 0.03497199,
       0.03037403, 0.02718236, 0.02417054, 0.02608138, 0.02786812,
       0.03042837, 0.03443333, 0.0386714 , 0.04372458, 0.04742577,
       0.05033314, 0.05153819, 0.05193154, 0.05137817, 0.05062424,
       0.0502719 , 0.0500798 , 0.04997239, 0.04892962, 0.04762865,
       0.04738085, 0.0478646 , 0.05066509, 0.05237981, 0.0519111 ,
       0.05268672, 0.05276742, 0.0519658 , 0.05005192, 0.05089552,
       0.04999307, 0.04933579, 0.0515698 , 0.0529885 , 0.05296163,
       0.05652263, 0.06062304, 0.06356298, 0.06656615, 0.0681862 ,
       0.0702354 , 0.07120796, 0.07127169, 0.07082662, 0.06964175,
       0.07278174, 0.06717502, 0.06766681, 0.06677362, 0.06720